<a href="https://colab.research.google.com/github/YahyaEryani/quantum-model/blob/main/notebooks/05_TabNet_model_training_and_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Training, Tuning, and Evaluation

 In this notebook, we will train an TabNet model on the Higgs boson dataset we have preprocessed in the `01_data_exploration` notebook. We will perform the model training and tuning process to obtain the best model with the highest accuracy possible.

## Installing and Importing Libraries
In this section, we will install and import the necessary libraries and packages that will be used throughout the notebook.

In [1]:
!pip install torch==1.10.0+cpu torchvision==0.11.1+cpu -f https://download.pytorch.org/whl/cpu/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/cpu/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.3/199.3 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 65.8 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0+cu118
    Uninstalling torch-2.0.0+cu118:
      Successfully uninstalled torch-2.0.0+cu118
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.15.1+cu118
    Uninstalling torchvision-0.15.1+cu118:
      Successfully uninstalled torchvision-0.15.1+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.15.1 requires torch==2.0.0, but you have torch 1.10.0+cpu which is incompatible.
torchdata 0.6.0 requires torch==2.0.0, 

In [2]:
!pip install pytorch-tabnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.5 MB/s eta 0:00:00


In [3]:
!pip install torch -U


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 85.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 7.5 MB/s eta 0:00:00
     ━━━━━

In [12]:
import torch
import numpy as np
import pandas as pd
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

## Loading Data

This code cell loads the training, validation, and test datasets that were saved in pickle format to the local directory.

In [5]:
# Mount Google Drive in Colab
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')

# Load data from Google Drive
train_path = '/content/drive/MyDrive/Higgs_dataset/processed/training_data.pkl'
val_path   = '/content/drive/MyDrive/Higgs_dataset/processed/validation_data.pkl'
test_path  = '/content/drive/MyDrive/Higgs_dataset/processed/testing_data.pkl'

train_data = pd.read_pickle(train_path)
val_data = pd.read_pickle(val_path)
test_data = pd.read_pickle(test_path)

Mounted at /content/drive


## Prepare the data for training
This code separates the features and class labels from the train, validation, and test datasets.

In [6]:
# Separate features and labels
y_train = train_data['class_label']
X_train = train_data.drop('class_label', axis=1)
y_val = val_data['class_label']
X_val = val_data.drop('class_label', axis=1)
y_test = test_data['class_label']
X_test = test_data.drop('class_label', axis=1)

##Train the TabNet model
This code sets the hyperparameters for a TabNet model, including the number of decision steps, the number of attention heads, learning rate, batch size, and number of epochs. It then trains the TabNet model using the Adam optimizer with a specified learning rate, batch size, and number of epochs. Additionally, the model performance is monitored using the validation set during training.

In [7]:
# Set the TabNet hyperparameters
tabnet_params = dict(
    n_d=64,
    n_a=64,
    n_steps=5,
    gamma=1.3,
    n_independent=2,
    n_shared=2,
    seed=42,
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=0.02),
    scheduler_params=dict(
        mode="min",
        patience=5,
        min_lr=1e-5,
        factor=0.9,
    ),
    scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
    mask_type="entmax",
    verbose=1
)

# Train the TabNet model
tabnet_model = TabNetClassifier(**tabnet_params)
tabnet_model.fit(
    X_train=X_train.values, y_train=y_train.values,
    eval_set=[(X_val.values, y_val.values)],
    max_epochs=100,
    patience=10,
    batch_size=256,
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False,
)

/usr/local/lib/python3.9/dist-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.62151 | val_0_auc: 0.76416 |  0:01:57s
epoch 1  | loss: 0.57629 | val_0_auc: 0.77859 |  0:03:54s
epoch 2  | loss: 0.56295 | val_0_auc: 0.78642 |  0:05:53s
epoch 3  | loss: 0.55643 | val_0_auc: 0.79139 |  0:08:00s
epoch 4  | loss: 0.54599 | val_0_auc: 0.80008 |  0:09:58s
epoch 5  | loss: 0.53801 | val_0_auc: 0.80774 |  0:11:57s
epoch 6  | loss: 0.53142 | val_0_auc: 0.80672 |  0:13:55s
epoch 7  | loss: 0.52765 | val_0_auc: 0.81273 |  0:15:54s
epoch 8  | loss: 0.52411 | val_0_auc: 0.81861 |  0:17:57s
epoch 9  | loss: 0.52227 | val_0_auc: 0.82037 |  0:19:56s
epoch 10 | loss: 0.51987 | val_0_auc: 0.81679 |  0:21:55s
epoch 11 | loss: 0.51796 | val_0_auc: 0.81806 |  0:23:53s
epoch 12 | loss: 0.51856 | val_0_auc: 0.82183 |  0:25:52s
epoch 13 | loss: 0.51542 | val_0_auc: 0.82313 |  0:27:57s
epoch 14 | loss: 0.51338 | val_0_auc: 0.82325 |  0:29:56s
epoch 15 | loss: 0.51177 | val_0_auc: 0.82552 |  0:31:55s
epoch 16 | loss: 0.51321 | val_0_auc: 0.82598 |  0:33:53s
epoch 17 | los

/usr/local/lib/python3.9/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [14]:
# Calculate the accuracy on the training set
y_train_pred = tabnet_model.predict(X_train.values)
train_accuracy = accuracy_score(y_train, y_train_pred)
print(f"Training Accuracy: {train_accuracy * 100:.2f}%")

Training Accuracy: 77.69%


## Make predictions on the test data and evaluate the model performance
This code uses the TabNet model that was previously trained to make predictions on the test data.

In [18]:
# Make predictions on the test data
y_test_pred = tabnet_model.predict(X_test.values)

# Calculate the accuracy of the model on the test data
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Test Accuracy: 74.82%
